In [ ]:
# Installing conda
!curl -Lo conda_installer.py https://raw.githubusercontent.com/deepchem/deepchem/master/scripts/colab_install.py
import conda_installer
conda_installer.install()
!/root/miniconda/bin/conda info -e

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3490  100  3490    0     0  14244      0 --:--:-- --:--:-- --:--:-- 14244


add /root/miniconda/lib/python3.6/site-packages to PYTHONPATH
python version: 3.6.9
fetching installer from https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
done
installing miniconda to /root/miniconda
done
installing rdkit, openmm, pdbfixer
added conda-forge to channels
added omnia to channels
done
conda packages installation finished!


# conda environments:
#
base                  *  /root/miniconda



In [ ]:
# Installing Deepchem
!pip install --pre deepchem
import deepchem
deepchem.__version__

'2.4.0-rc1.dev'

In [ ]:
# Getting the dataset
dataset_file= "Log_Clearance.csv"
from deepchem.utils.save import load_from_disk
dataset_file= "Log_Clearance.csv"

# Loading the data from the CSV file
loader = deepchem.data.CSVLoader(tasks=["Log_Cl"], 
                                 feature_field="SMILES", 
                                 featurizer=deepchem.feat.ConvMolFeaturizer())
# Featurizing the dataset with ConvMolFeaturizer
dataset = loader.featurize(dataset_file)

/usr/local/lib/python3.6/dist-packages/deepchem/data/data_loader.py:162: FutureWarning: featurize() is deprecated and has been renamed to create_dataset().featurize() will be removed in DeepChem 3.0
  "featurize() will be removed in DeepChem 3.0", FutureWarning)
Failed to featurize datapoint 165, None. Appending empty array
Exception message: Python argument types in
    rdkit.Chem.rdmolfiles.CanonicalRankAtoms(NoneType)
did not match C++ signature:
    CanonicalRankAtoms(RDKit::ROMol mol, bool breakTies=True, bool includeChirality=True, bool includeIsotopes=True)
Failed to featurize datapoint 481, None. Appending empty array
Exception message: Python argument types in
    rdkit.Chem.rdmolfiles.CanonicalRankAtoms(NoneType)
did not match C++ signature:
    CanonicalRankAtoms(RDKit::ROMol mol, bool breakTies=True, bool includeChirality=True, bool includeIsotopes=True)
Failed to featurize datapoint 482, None. Appending empty array
Exception message: Python argument types in
    rdkit.Chem

In [ ]:
    # Splitter splits the dataset 
    splitter = deepchem.splits.RandomSplitter()
    
    train, valid, _ = splitter.train_valid_test_split(dataset,
                                                      frac_train=0.7,
                                                      frac_valid=0.29,
                                                      frac_test=0.01)
    # Normalizer will normalize y values in the dataset
    normalizer = deepchem.trans.NormalizationTransformer(transform_y=True, 
                                                         dataset=train, 
                                                         move_mean=True)
    train = normalizer.transform(train)
    test = normalizer.transform(valid)

In [ ]:
print(f"Size of the training data: {len(train.ids)}")
print(f"Size of the validation data: {len(valid.ids)}")
print(test)

Size of the training data: 830
Size of the validation data: 345
<DiskDataset X.shape: (345,), y.shape: (345, 1), w.shape: (345, 1), ids: ['C(CCNCCN1C(=O)c2c(ccc3ccccc23)C1=O)NCCN1C(=O)c2c(ccc3ccccc23)C1=O'
 'n1(nnnc1SCC1=C(N2[C@H](OC1)[C@@](C2=O)(OC)NC(=O)CSC(F)F)C(O)=O)CCO'
 'CC(C)C[C@@H]1CN2CCc3cc(c(cc3[C@H]2C[C@H]1OC(=O)[C@H](C(C)C)N)OC)OC' ...
 'COc1c(ccc2c1N=C(N3C2=NCC3)NC(=O)c4cnc(nc4)N)OCCCN5CCOCC5' 'NCCCCCC(=O)O'
 'CCOC(=O)C1(CCN(C)CC1)c2ccccc2'], task_names: ['Log_Cl']>


In [ ]:
# GraphConvModel is a GNN model based on Duvenaud, David K., et al. "Convolutional networks on graphs for learning molecular fingerprints."
from deepchem.models import GraphConvModel
graph_conv = GraphConvModel(1,
                            batch_size=50,
                            mode="regression")

metric = deepchem.metrics.Metric(deepchem.metrics.pearson_r2_score)


In [ ]:
# Fitting the model
graph_conv.fit(train, nb_epoch=10)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


0.3239185060773577

In [ ]:
# Reversing the transformation and getting the metric scores on 2 datasets
train_scores = graph_conv.evaluate(train, [metric], [normalizer])
valid_scores = graph_conv.evaluate(valid, [metric], [normalizer])
print(f"Train Scores: {train_scores}")
print(f"Validation Scores: {valid_scores}")

Train Scores: {'pearson_r2_score': 0.2706687444703349}
Validation Scores: {'pearson_r2_score': 0.0836620565725482}
